# 공유 자전거 수요 예측 (11.30 고지혜)

### 라이브러리 및 데이터 불러오기

In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd 

# 시각화 라이브러리
import matplotlib.pyplot as plt 
import seaborn as sns

# 모델링을 위한 sklearn 패키지
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# score를 내줄 함수
from sklearn.metrics import make_scorer

# 모델링에 활용한 패키지
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
train = pd.read_csv('prepro_train.csv')
train.head()

,datetime,holiday,workingday,atemp,humidity,windspeed,log_count,rainyday,ideal,sticky,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2011-01-01 00:00:00,0,0,0.305068,0.81,0.219409,2.833213,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2011-01-01 01:00:00,0,0,0.288064,0.80,0.219409,3.713572,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2011-01-01 02:00:00,0,0,0.288064,0.80,0.219409,3.496508,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2011-01-01 03:00:00,0,0,0.305068,0.75,0.219409,2.639057,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2011-01-01 04:00:00,0,0,0.305068,0.75,0.219409,0.693147,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test = pd.read_csv('prepro_test.csv')
test.tail()

,datetime,holiday,workingday,atemp,humidity,windspeed,rainyday,ideal,sticky,peak,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
6488,2012-12-31 19:00:00,0,1,0.2576,0.523810,0.099973,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6489,2012-12-31 20:00:00,0,1,0.2576,0.523810,0.099973,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6490,2012-12-31 21:00:00,0,1,0.2576,0.523810,0.099973,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6491,2012-12-31 22:00:00,0,1,0.2727,0.476190,0.059904,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6492,2012-12-31 23:00:00,0,1,0.2727,0.583333,0.059904,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 베이스라인 모델링
특정 기법을 통해 학습 및 평가했을 때, 기존의 모델보다 좋아졌는지 판단하기 위해 기준으로 삼을 베이스라인 모델을 생성한다.

In [4]:
# 전처리한 변수들이 있는데 이 중에서 분석에 활용한 변수를 선택해줍시다.
# 기존 데이터에 덮어쓰기보단 train_copy라는 예비 데이터프레임을 생성하여 저장해줍시다.
train_copy = train

col = ['holiday', 'workingday', 'atemp', 'humidity', 'windspeed',
       'rainyday', 'ideal', 'sticky', 'peak', 'temp(difference)',
       'discomfort_index', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hw_0', 'hw_1',
       'hw_2', 'hw_3', 'hw_5', 'hw_6', 'hw_7', 'hw_8', 'hw_9', 'hw_10',
       'hw_11', 'hw_12', 'hw_13', 'hw_14', 'hw_15', 'hw_16', 'hw_17', 'hw_18',
       'hw_19', 'hw_20', 'hw_21', 'hw_22', 'hw_23', 'year_2012', 'dayofweek_0',
       'dayofweek_2', 'dayofweek_3', 'dayofweek_4', 'dayofweek_5',
       'dayofweek_6', 'season_2', 'season_3', 'season_4', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12']

# count를 제외한 변수들을 담은 데이터프레임.
X_features = train[col]
X_test = test[col]

# 타겟 변수는 log 처리를 해준 count 변수
target = train['log_count']

# 데이터를 나눠줌
X_train, X_valid, y_train, y_valid = train_test_split(X_features, target, test_size = 0.3, random_state = 0)

In [5]:
train.columns

Index(['datetime', 'holiday', 'workingday', 'atemp', 'humidity', 'windspeed',
       'log_count', 'rainyday', 'ideal', 'sticky', 'peak', 'temp(difference)',
       'discomfort_index', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hw_0', 'hw_1',
       'hw_2', 'hw_3', 'hw_5', 'hw_6', 'hw_7', 'hw_8', 'hw_9', 'hw_10',
       'hw_11', 'hw_12', 'hw_13', 'hw_14', 'hw_15', 'hw_16', 'hw_17', 'hw_18',
       'hw_19', 'hw_20', 'hw_21', 'hw_22', 'hw_23', 'year_2012', 'dayofweek_0',
       'dayofweek_2', 'dayofweek_3', 'dayofweek_4', 'dayofweek_5',
       'dayofweek_6', 'season_2', 'season_3', 'season_4', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12'],
      dtype='object')

In [6]:
# RMSLE 값을 출력하는 함수
def rmsle(y,y_,convertExp=True):
    # 지수화 필요하다면
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [7]:
## cross val score를 측정해주는 함수
def cv_score(models, n = 5):
    # kfold 수는 default인 5로 지정
    kf = KFold(n_splits = n, shuffle=True)
    
    for model in models:
#       model.fit(X_train,y_train)
        score =  cross_val_score(model, X_features, target, cv = kf, scoring=rmsle_scorer)
        print(model[0],'의 평균 score:', round(score.mean(), 5))
        print(model[0],'의 평균 std:', round(score.std(), 5))
        print()      
        
        # y_valid과 prediction을 비교하여 시각화 해주는 코드
#        g = sns.kdeplot(np.exp(y_valid),  color = 'skyblue', alpha = .6, fill = True, label = 'valid')
#        g = sns.kdeplot(np.exp(model.predict(X_valid)), color = 'orange', alpha = .3, fill = True, label = 'prediction')
#       plt.legend()
#        plt.show()

In [7]:
## 제출을 위한 함수
def submission(model):
    model.fit(X_features, target)
    prediction = np.exp(model.predict(X_test))
    
    # 자동으로 형식을 맞춰 csv 생성해주는 코드
    submission = pd.DataFrame(test['datetime'])
    submission['count'] = prediction
    pd.DataFrame(submission).to_csv('submission_bike.csv', index = False)
    
    return pd.DataFrame(submission)

In [9]:
from sklearn.linear_model import ElasticNet
# 기본 모델을 아래와 같이 5가지로 정했음.
pipe_lr = Pipeline([('model', LinearRegression())])
pipe_rf = Pipeline([('model', RandomForestRegressor(n_estimators=500))])
pipe_lgbm = Pipeline([('model', LGBMRegressor(n_estimators=100))])
pipe_gb = Pipeline([('model', GradientBoostingRegressor())])
pipe_ela = Pipeline([('model',ElasticNet())])

models = [pipe_lr, pipe_rf, pipe_lgbm, pipe_gb, pipe_ela]

# 평균 score 측정
cv_score(models)

LinearRegression() 의 평균 score: 0.33559
LinearRegression() 의 평균 std: 0.00502

RandomForestRegressor(n_estimators=500) 의 평균 score: 0.43999
RandomForestRegressor(n_estimators=500) 의 평균 std: 0.01454

LGBMRegressor() 의 평균 score: 0.32053
LGBMRegressor() 의 평균 std: 0.00611

GradientBoostingRegressor() 의 평균 score: 0.59617
GradientBoostingRegressor() 의 평균 std: 0.01196

ElasticNet() 의 평균 score: 1.36844
ElasticNet() 의 평균 std: 0.02458



In [10]:
from xgboost import XGBRegressor

pipe_xgb = Pipeline([('model', XGBRegressor(objective ='reg:squarederror'))])

kf = KFold(n_splits = 5, shuffle=True)

# pipe_xgb.fit(X_train,y_train)
score =  cross_val_score(pipe_xgb, X_features, target, cv = kf, scoring=rmsle_scorer)

In [11]:
score.mean()   # 0.39756

0.3962258595528969

In [12]:
# lr 모델 제출 결과 : => inf 값 나와서 실패.. 왜지...
# rf 모델 제출 결과 : 0.52784  **
# lgbm 모델 제출 결과 : 0.41477  **
# gb 모델 제출 결과 : 0.66858
# ela 모델 제출 결과 : 1.41582
# xgb 모델 제출 결과 : 0.52387  **
submission(pipe_xgb)

,datetime,count
0,2011-01-20 00:00:00,11.115300
1,2011-01-20 01:00:00,7.396193
2,2011-01-20 02:00:00,4.972657
3,2011-01-20 03:00:00,2.844000
4,2011-01-20 04:00:00,13.532303
...,...,...
6488,2012-12-31 19:00:00,232.311264
6489,2012-12-31 20:00:00,143.047028
6490,2012-12-31 21:00:00,105.450508
6491,2012-12-31 22:00:00,69.192413


[RMSLE 평가 지표에 대해](https://ahnjg.tistory.com/90)

요약 
1. 큰 것보다 적은 것을 오차없이 예측할 때 점수가 더 좋음.
2. under estimator에 대해 페널티를 부과한다. <b> 예측값 > 실제값</b> 보다 <b>예측값 < 실제값</b>일 때, 점수가 안 좋음.

## rf + lgbm + xgb 앙상블해보기

## 파라미터 찾기

def search_paras(x, y, model, paras, n = 10, scorer = rmsle_scorer) :
    grid_model = GridSearchCV(estimator = model, param_grid = paras, cv=n, n_jobs=-1, verbose=2, scoring = scorer)
    grid_model.fit(x,y)

    #lgbm 
    print(model," best param is : ", grid_model.best_params_)
    print(model," best score is : ", grid_model.best_score_)

In [15]:
#lgbm에 대한 parameters
para_lgbm = [{
    'learning_rate' : [0.01, 0.03, 0.05, 0.07, 0.1],
    'n_estimators' : [500, 800, 1000, 1300, 1500]}]

In [16]:
grid_lgbm = LGBMRegressor(random_state = 0)

grid_lgbm = GridSearchCV(estimator = grid_lgbm, param_grid = para_lgbm, cv=10, n_jobs=-1, verbose=2, scoring = rmsle_scorer)
grid_lgbm.fit(X_features, target)

#lgbm 
print("lgbm best param is : ", grid_lgbm.best_params_)
print("lgbm best score is : ", grid_lgbm.best_score_)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
lgbm best param is :  {'learning_rate': 0.01, 'n_estimators': 500}
lgbm best score is :  0.43290658428538464


In [17]:
pipe_lgbm2 = Pipeline([('model', LGBMRegressor(n_estimators=500, learning_rate = 0.01))])

submission(pipe_lgbm2) # 0.46298

,datetime,count
0,2011-01-20 00:00:00,14.026866
1,2011-01-20 01:00:00,6.733000
2,2011-01-20 02:00:00,4.735953
3,2011-01-20 03:00:00,3.552059
4,2011-01-20 04:00:00,5.996315
...,...,...
6488,2012-12-31 19:00:00,149.577223
6489,2012-12-31 20:00:00,105.427349
6490,2012-12-31 21:00:00,77.934665
6491,2012-12-31 22:00:00,72.446662


In [18]:
pipe_lgbm3 = Pipeline([('model', LGBMRegressor(n_estimators=1000, learning_rate = 0.05))])
submission(pipe_lgbm3)  # 0.396

,datetime,count
0,2011-01-20 00:00:00,11.975283
1,2011-01-20 01:00:00,5.566891
2,2011-01-20 02:00:00,5.269114
3,2011-01-20 03:00:00,2.867136
4,2011-01-20 04:00:00,2.599130
...,...,...
6488,2012-12-31 19:00:00,250.991344
6489,2012-12-31 20:00:00,152.620581
6490,2012-12-31 21:00:00,101.231304
6491,2012-12-31 22:00:00,75.001111


In [22]:
para_xgb = [{
    'eta' : [0.01, 0.05, 0.1, 0.15, 0.2], 
    'gamma' : [0, 0.3, 0.5],
    'max_depth': [4, 6, 8],
    'random_state' : [0],
    'objective' : ['reg:squarederror']}]

grid_xgb = XGBRegressor()


xgb_para = GridSearchCV(estimator = grid_xgb, param_grid = para_xgb, cv=10, n_jobs=-1, verbose=2, scoring = rmsle_scorer)
xgb_para.fit(X_features, target)

Fitting 10 folds for each of 45 candidates, totalling 450 fits


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs...
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parame

In [25]:
print("xgb best param is : ", xgb_para.best_params_)
print("xgb best score is : ", xgb_para.best_score_)

xgb best param is :  {'eta': 0.01, 'gamma': 0.3, 'max_depth': 4, 'objective': 'reg:squarederror', 'random_state': 0}
xgb best score is :  1.770375056016508


In [29]:
pipe_xgb2 = Pipeline([('model', XGBRegressor(eta = 0.01, gamma= 0.3, max_depth= 4, objective= 'reg:squarederror', random_state= 0))])
submission(pipe_xgb2)  # 1.7656

,datetime,count
0,2011-01-20 00:00:00,12.940742
1,2011-01-20 01:00:00,8.886965
2,2011-01-20 02:00:00,5.866405
3,2011-01-20 03:00:00,4.126398
4,2011-01-20 04:00:00,12.940742
...,...,...
6488,2012-12-31 19:00:00,15.111762
6489,2012-12-31 20:00:00,13.103193
6490,2012-12-31 21:00:00,13.103193
6491,2012-12-31 22:00:00,13.103193


#rf에 대한 parameters
para_rf = [{
    'max_depth' : [6, 8, 10, 12, 14],
    'n_estimators' : [100,200,300,400,500],
    'min_samples_split' : [2, 5, 7,10],
    'min_samples_leaf' : [1, 2, 4],
    'random_state' : [0]}]

grid_rf = RandomForestRegressor()

rf_para = GridSearchCV(estimator = grid_rf, param_grid = para_rf, cv=10, n_jobs=-1, verbose=2, scoring = rmsle_scorer)
rf_para.fit(X_features, target)

#lgbm 
print("rf best param is : ", rf_para.best_params_)
print("rf best score is : ", rf_para.best_score_)